In [ ]:
import requests
import _pickle as cPickle
import shutil
import time

## 1. Retrieve data from the Premier League API and save locally

In [ ]:
outfile = open("players2017.data.pickle", "wb")

In [ ]:
# Retrieve data from the web API and store player data in a pickle file

# Note: the max number of players expected might need to be changed depending on season
MAX_NUMBER_PLAYERS = 700

all = {}

for i in range(MAX_NUMBER_PLAYERS):
    playerurl = "http://fantasy.premierleague.com/drf/element-summary/%s"
    r = requests.get(playerurl % i)

    # skip non-existent players
    if r.status_code != 200: continue

    try:
        all[i] = r.json()
    except ValueError:
        continue
        

In [ ]:
cPickle.dump(all, outfile)

In [ ]:
outfile.close()

In [ ]:
# Pickle additional data

outfile = open("static2017.data.pickle", "wb")
staticurl = "https://fantasy.premierleague.com/drf/bootstrap-static"
r = requests.get(staticurl)
staticdata = r.json()
cPickle.dump(staticdata, outfile)
outfile.close()

## 2. Load from pickle file

In [ ]:
# Load player data from the pickle file

players = cPickle.load(open("players2017.data.pickle","rb"))

static = cPickle.load(open("static2017.data.pickle","rb"))

teams = static["teams"]

positions = static["element_types"]


## 3. Merge data into one table and save to CSV

In [ ]:
import csv

with open("playerdata.csv", "w") as file:
    csv_file = csv.writer(file)
    csv_file.writerow(list(static["elements"][0].keys())
                      +["type_"+k for k in list(static["element_types"][0].keys())]
                      +["team_"+k for k in list(static["teams"][0].keys())]
                      #+["fixtures_no"]
                      #+players[1].keys()
                     ) 
    for i,element in enumerate(static["elements"]):
        vals = []
        for v in element.values():
            if isinstance(v, str): vals.append(v.encode('utf-8'))
            else: vals.append(v)
        playertype = positions[element["element_type"]-1]
        for p in playertype.values():
            if isinstance(p, str): vals.append(p.encode('utf-8'))
            else: vals.append(p)
        teaminfo = teams[element["team"]-1]
        for t in teaminfo.values():
            if isinstance(t, str): vals.append(t.encode('utf-8'))
            else: vals.append(t)
        #vals.append(str(len(players[i+1]["fixtures"])))
        #for param in players[i+1].values():
        #    if isinstance(param, basestring): vals.append(param.encode('utf-8'))
        #    else: vals.append(param)
        csv_file.writerow(vals)